In [0]:
import numpy as np

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
# MNIST Dataset
original_train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
original_test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 19181562.66it/s]                            


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 296707.89it/s]                           
0it [00:00, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5253703.09it/s]                           
8192it [00:00, 117440.68it/s]


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
CUDA = True
BATCH_SIZE = 64

In [0]:
# Define Train loader
train_tensors = original_train_dataset.data.float() / 255
test_tensors = original_test_dataset.data.float() / 255

train_dataset = torch.utils.data.TensorDataset(train_tensors, original_train_dataset.targets)
test_dataset = torch.utils.data.TensorDataset(test_tensors, original_test_dataset.targets)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [0]:
discriminator = nn.Sequential(
    nn.Dropout(),
    nn.Linear(784, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 128),
    nn.LeakyReLU(0.2),
    nn.Linear(128, 64),
    nn.Dropout(),
    nn.LeakyReLU(0.2),
    nn.Linear(64, 1),
    nn.Sigmoid()
)

In [0]:
generator = nn.Sequential(
    nn.Linear(128, 128),
    nn.LeakyReLU(0.2),
    nn.Linear(128, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 512),
    nn.LeakyReLU(0.2),
    nn.Linear(512, 784),
    nn.Sigmoid()
)

In [0]:
if CUDA:
  discriminator.cuda()
  generator.cuda()

In [0]:
LEARNING_RATE_D = 0.0002
LEARNING_RATE_G = 0.0002


opt_D = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE_D)
opt_G = optim.Adam(generator.parameters(), lr=LEARNING_RATE_G)

In [0]:
N_EPOCH = 100

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 3) # set default size of plots

In [12]:
# [CHANGED] EPSILON is not needed anymore
# EPSILON = 1e-6

for epoch in range(N_EPOCH):
    for i, (img, label) in enumerate(train_loader):
        img = img.flatten(start_dim=1)

        real_img = img
        if CUDA:
            real_img = real_img.cuda()

        fake_labels = torch.zeros(img.shape[0], 1).cuda()
        real_labels = torch.ones(img.shape[0], 1).cuda()

        z = torch.randn(img.shape[0], 128)
        if CUDA:
            z = z.cuda()
        fake_img = generator(z)

        opt_D.zero_grad()
        loss_d = (F.binary_cross_entropy(discriminator(fake_img.detach()), fake_labels) +
                  F.binary_cross_entropy(discriminator(real_img), real_labels))
        loss_d.backward()
        opt_D.step()

        z = torch.randn(img.shape[0], 128)
        if CUDA:
          z = z.cuda()
        fake_img = generator(z)

        opt_G.zero_grad()
        loss_g = F.binary_cross_entropy(discriminator(fake_img), real_labels)
        loss_g.backward()
        opt_G.step()

    
    print("epoch: {} \t last batch loss D: {} \t last batch loss G: {}".format(epoch, loss_d.item(), loss_g.item()))

    for i in range(3):
        for j in range(10):
            plt.subplot(3, 10, i * 10 + j + 1)
            plt.imshow(fake_img[i * 10 + j].detach().cpu().view(28, 28).numpy())
    plt.show()



Output hidden; open in https://colab.research.google.com to view.

In [0]:
discriminator_for_linear_initialization = nn.Sequential(
    nn.Dropout(),
    nn.Linear(784, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 128),
    nn.LeakyReLU(0.2),
    nn.Linear(128, 64),
    nn.Dropout(),
    nn.LeakyReLU(0.2),
    nn.Linear(64, 1),
    nn.Sigmoid()
)

generator_for_linear_initialization = nn.Sequential(
    nn.Linear(128, 128),
    nn.LeakyReLU(0.2),
    nn.Linear(128, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 512),
    nn.LeakyReLU(0.2),
    nn.Linear(512, 784),
    nn.Sigmoid()
)

if CUDA:
  discriminator_for_linear_initialization.cuda()
  generator_for_linear_initialization.cuda()

  
LEARNING_RATE_D_for_linear_initialization = 0.0002
LEARNING_RATE_G_for_linear_initialization = 0.0002


opt_D = optim.Adam(discriminator_for_linear_initialization.parameters(), 
                   lr=LEARNING_RATE_D_for_linear_initialization)
opt_G = optim.Adam(generator_for_linear_initialization.parameters(), 
                   lr=LEARNING_RATE_G_for_linear_initialization)

In [14]:
for epoch in range(N_EPOCH):
    for i, (img, label) in enumerate(train_loader):
        img = img.flatten(start_dim=1)

        real_img = img
        if CUDA:
            real_img = real_img.cuda()

        fake_labels = torch.zeros(img.shape[0], 1).cuda()
        real_labels = torch.ones(img.shape[0], 1).cuda()

        z = torch.rand(img.shape[0], 128) * 2 - 1
        if CUDA:
            z = z.cuda()
        fake_img = generator_for_linear_initialization(z)

        opt_D.zero_grad()
        loss_d = (F.binary_cross_entropy(discriminator_for_linear_initialization(fake_img.detach()), fake_labels) +
                  F.binary_cross_entropy(discriminator_for_linear_initialization(real_img), real_labels))
        loss_d.backward()
        opt_D.step()
        
        z = torch.rand(img.shape[0], 128) * 2 - 1
        if CUDA:
          z = z.cuda()
        fake_img = generator_for_linear_initialization(z)

        opt_G.zero_grad()
        loss_g = F.binary_cross_entropy(discriminator_for_linear_initialization(fake_img), real_labels)
        loss_g.backward()
        opt_G.step()

    
    print("epoch: {} \t last batch loss D: {} \t last batch loss G: {}".format(epoch, loss_d.item(), loss_g.item()))

    for i in range(3):
        for j in range(10):
            plt.subplot(3, 10, i * 10 + j + 1)
            plt.imshow(fake_img[i * 10 + j].detach().cpu().view(28, 28).numpy())
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [0]:
N_EPOCH = 100
d = 32
discriminator_DC = nn.Sequential(
    nn.Conv2d(1, d, 4, 2, 1), #14
    nn.LeakyReLU(0.2),
    nn.Conv2d(d, d*2, 4, 2, 1), #7
    nn.BatchNorm2d(d*2),
    nn.LeakyReLU(0.2),
    nn.Conv2d(d*2, d*4, 4, 2, 1), #4 
    nn.BatchNorm2d(d*4),
    nn.LeakyReLU(0.2),
    nn.Conv2d(d*4, 1, 3, 2, 0), #1 
    nn.Sigmoid()
)

generator_DC = nn.Sequential(
    nn.ConvTranspose2d(100, d*8, 3, 2, 0), #3
    nn.BatchNorm2d(d*8),
    nn.ReLU(),
    nn.ConvTranspose2d(d*8, d*4, 3, 2, 0), #7
    nn.BatchNorm2d(d*4),
    nn.ReLU(),
    nn.ConvTranspose2d(d*4, d*2, 3, 2, 0), # 15
    nn.BatchNorm2d(d*2),
    nn.ReLU(),
    nn.ConvTranspose2d(d*2, 1, 2, 2, 1), #28
)


if CUDA:
  discriminator_DC.cuda()
  generator_DC.cuda()


  
LEARNING_RATE_D_DC = 0.002
LEARNING_RATE_G_DC = 0.002

opt_D = optim.Adam(discriminator_DC.parameters(), 
                   lr=LEARNING_RATE_D_DC)
opt_G = optim.Adam(generator_DC.parameters(), 
                   lr=LEARNING_RATE_G_DC)

In [16]:
for epoch in range(N_EPOCH):
    for i, (img, label) in enumerate(train_loader):
      
        real_img = img.view(-1, 1, 28, 28)
        if CUDA:
            real_img = real_img.cuda()

        fake_labels = torch.zeros(img.shape[0], 1,1,1).cuda()
        real_labels = torch.ones(img.shape[0], 1,1,1).cuda()
#         noise = torch.randn(img.shape[0], 28 * 28).view(-1, 1, 28, 28)
#         if CUDA:
#           noise = noise.cuda()
        real_img 
    
        z = torch.randn(img.shape[0], 100)
        z = z.view(-1, 100, 1, 1)
        if CUDA:
            z = z.cuda()
        fake_img = generator_DC(z) 
        opt_D.zero_grad()
        loss_d = (F.binary_cross_entropy(discriminator_DC(fake_img.detach()), 
                                         fake_labels) 
                  + F.binary_cross_entropy(discriminator_DC(real_img), 
                                           real_labels))
        loss_d.backward()
        opt_D.step()
        
        z = torch.randn(img.shape[0], 100)
        z = z.view(-1, 100, 1, 1)
        
        if CUDA:
          z = z.cuda()
        fake_img = generator_DC(z)
        
        opt_G.zero_grad()
        loss_g = F.binary_cross_entropy(discriminator_DC(fake_img), real_labels)
        loss_g.backward()
        opt_G.step()
        

    
    print("epoch: {} \t last batch loss D: {} \t last batch loss G: {}".format(epoch, loss_d.item(), loss_g.item()))

    for i in range(3):
        for j in range(10):
            plt.subplot(3, 10, i * 10 + j + 1)
            plt.imshow(fake_img[i * 10 + j].detach().cpu().view(28, 28).numpy())
    plt.show()

Output hidden; open in https://colab.research.google.com to view.